# Myers Briggs Personality Type Classification - Abridged

### Carolina Analytics and Data Science

Data from [Kaggle](https://www.kaggle.com/datasnaek/mbti-type)

The goal for this dataset will to be able to classify Myers Briggs Personality Type (MBTI) based on text someone with that personality type had written. In our meeting on Febuary 15 we will choose which 

This is starter code to get working on this dataset.

Note there are two versions of this document. In this (the "abridged") we start with already featurized text that we have made here so that jumping right into the modelling is also an option. 

In [1]:
import tensorflow as tf # neural net library
import numpy as np      # numerical/linear algebra
import pandas as pd     # reading in data
import re               # regular expression
import wordcloud
import csv

We use the CSV's generated in the "full" version

In [2]:
full_training_featurized = pd.read_csv('data/featurized_train.csv', index_col='instance')
featurized_testing = pd.read_csv('data/featurized_test.csv', index_col='instance')

## Prediction

We decided in our meeting to predict Thinking and Feeling, so the third letter of the classification.

Splitting into test and validation sets

In [3]:
full_training_featurized['Y'] = (1*(full_training_featurized['tf'] == 'T')) # having a Y column that is 0-1 valued

np.random.seed(seed=1) # for consistency with test and training sets
training_data, validation_data = np.split(full_training_featurized.sample(frac=1), [int(.75*len(full_training_featurized))]) # splitting into training and validation sets

### Logistic Regression

In [4]:
training_x = training_data.drop(['ie', 'sn', 'tf', 'pj', 'Y'], axis=1)
training_y = training_data['Y']

validation_x = validation_data.drop(['ie', 'sn', 'tf', 'pj', 'Y'], axis=1)
validation_y = validation_data['Y']

from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg = logreg.fit(training_x, training_y)

logreg.score(training_x, training_y) # training accuracy

0.5394812680115274

In [5]:
def evaluate_model(model, validation_x, validation_y): 
    # testing accuracy 
    correct = 0
    for (a, b) in zip(model.predict(validation_x), validation_y):
        correct += (a-b)**2
    return float(correct)/len(validation_y)

evaluate_model(logreg, validation_x, validation_y)

0.454178674351585

### Random Forest

In [6]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()
random_forest = random_forest.fit(training_x, training_y)

random_forest.score(training_x, training_y) # training accuracy

0.98405379442843421

In [7]:
evaluate_model(random_forest, validation_x, validation_y) # testing accuracy 

0.44726224783861673

### Tensorflow Neural Net

In [8]:
tf_training_y = np.array([training_y, -(training_y-1)]).T
tf_validation_y = np.array([validation_y, -(validation_y-1)]).T
# Parameters
learning_rate = 0.001
training_epochs = 1000
batch_size = 100
display_step = 10


# Network Parameters
n_hidden_1 = 30 # 1st layer number of features
n_hidden_2 = 20 # 2nd layer number of features
n_input = 50 # Number of feature
n_classes = 2 # Number of classes to predict

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [9]:
# Launch the graph

with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(training_x)/batch_size)
        X_batches = np.array_split(training_x, total_batch)
        Y_batches = np.array_split(tf_training_y, total_batch)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Testing Accuracy:", accuracy.eval({x: validation_x, y: tf_validation_y}))
    global result 
    result = tf.argmax(pred, 1).eval({x: validation_x, y: tf_validation_y})

Epoch: 0001 cost= 1.023678432
Epoch: 0011 cost= 0.673817996
Epoch: 0021 cost= 0.659029766
Epoch: 0031 cost= 0.653910299
Epoch: 0041 cost= 0.651840323
Epoch: 0051 cost= 0.650847121
Epoch: 0061 cost= 0.650112472
Epoch: 0071 cost= 0.649528487
Epoch: 0081 cost= 0.649032772
Epoch: 0091 cost= 0.648469256
Epoch: 0101 cost= 0.647889245
Epoch: 0111 cost= 0.647322127
Epoch: 0121 cost= 0.646470057
Epoch: 0131 cost= 0.645499993
Epoch: 0141 cost= 0.644632240
Epoch: 0151 cost= 0.643568308
Epoch: 0161 cost= 0.642698663
Epoch: 0171 cost= 0.641924907
Epoch: 0181 cost= 0.641100556
Epoch: 0191 cost= 0.640194222
Epoch: 0201 cost= 0.639317993
Epoch: 0211 cost= 0.638550467
Epoch: 0221 cost= 0.637822088
Epoch: 0231 cost= 0.637159235
Epoch: 0241 cost= 0.636580355
Epoch: 0251 cost= 0.636020049
Epoch: 0261 cost= 0.635442792
Epoch: 0271 cost= 0.634940700
Epoch: 0281 cost= 0.634449125
Epoch: 0291 cost= 0.633768646
Epoch: 0301 cost= 0.633194850
Epoch: 0311 cost= 0.632712438
Epoch: 0321 cost= 0.632157267
Epoch: 033

### Formatting submission
Submission should be a csv with one column labeled "instance" and the other "tf" which should contain the prediction of "T" or "F"

In [10]:
_predictions = random_forest.predict(featurized_testing)
predictions = featurized_testing.reset_index()[['instance']]
predictions['tf'] = np.array(list(map(lambda x: 'T' if x == 1 else 'F', _predictions)))
predictions.to_csv('random_forest_example_submission.csv')